**FILTRADO de SNPs - MUESTRAS ILLUMINA:**

In [ ]:
import pandas as pd
import os
import glob
from datetime import datetime

In [ ]:
main_directory = "/home/laura/andrea/scripts/Samples_Script_Test"

# Carpeta de salida para el archivo con los SNPs filtrados
output_directory = "/home/laura/andrea/scripts/Filtered_SNPs_TB_DDBB"
os.makedirs(output_directory, exist_ok=True)  # Crea la carpeta si no existe

# DataFrame final BBDD_TB con las columnas de interés
df_BBDD_TB = pd.DataFrame(columns=['REGION', 'POS', 'REF', 'ALT', 'SAMPLES'])

# Búsqueda de archivos TSV en todas las subcarpetas
for folder_path, subfolders, files in os.walk(main_directory):

    if os.path.isdir(folder_path):
        tsv_files = glob.glob(os.path.join(folder_path, "snps.all.ivar.tsv"))  # Busca archivos en cada subcarpeta con el patrón especificado

        for file in tsv_files:
            try:
                # Cargar el archivo en un DataFrame
                df = pd.read_csv(file, sep='\t')

                # FILTRADO por TOTAL_DP y ALT_FREQ
                df_filtered = df[(df['TOTAL_DP'] >= 20) & (df['ALT_FREQ'] >= 0.7)]

                # FILTRADO de SNPs, se quitan las INDELs
                df_filtered = df_filtered[(df_filtered['TYPE'] == "snp")]

                # Eliminar columnas innecesarias
                df_filtered = df_filtered.drop(columns=['ID', 'ALT_QUAL', 'FILTER', 'TOTAL_DP', 'TYPE', 'REF_DP', 
                                                        'ALT_DP', 'REF_FREQ', 'ALT_FREQ', 'OLDVAR'], errors = 'ignore')

                # Obtener el nombre de la muestra
                sample_name = os.path.basename(folder_path)

                # Comparar cada SNP con los ya existentes en df_BBDD_TB
                for _, snp in df_filtered.iterrows():
                    mask = (
                        (df_BBDD_TB['POS'] == snp['POS']) & 
                        (df_BBDD_TB['REF'] == snp['REF']) & 
                        (df_BBDD_TB['ALT'] == snp['ALT'])
                    )

                    if mask.any():
                        # Obtener el índice del SNP existente
                        index = df_BBDD_TB.loc[mask].index[0]

                        # Agregar nombre de la muestra si no está ya presente
                        existing_samples = df_BBDD_TB.at[index, 'SAMPLES']
                        if sample_name not in existing_samples.split(", "):  # Evita duplicados
                            df_BBDD_TB.at[index, 'SAMPLES'] += f", {sample_name}"
                    else:
                        # SNP nuevo, añadirlo con la muestra asignada
                        new_snp = snp.copy()
                        new_snp['SAMPLES'] = sample_name
                        df_BBDD_TB = pd.concat([df_BBDD_TB, pd.DataFrame([new_snp])], ignore_index=True)

                print(f'Procesado: {file} -> SNPs añadidos a BBDD_TB')

            except Exception as e:
                print(f'Error procesando {file}: {e}')

# Calcular COUNT como la cantidad de muestras en 'SAMPLES'
df_BBDD_TB['COUNT'] = df_BBDD_TB['SAMPLES'].apply(lambda x: len(x.split(", ")))

# Ordenar los IDs de las muestras de la columna 'SAMPLES'
df_BBDD_TB['SAMPLES'] = df_BBDD_TB['SAMPLES'].apply(lambda x: ', '.join(sorted(set(x.split(', ')))))

# Ordenar SNPs por posición en el genoma de TB
df_BBDD_TB.sort_values(by = ['POS'], inplace = True)

# Generar nombre del archivo de salida
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = os.path.join(output_directory, f"Filtered_SNPs_{timestamp}.tsv")

# Guardar el DataFrame final
df_BBDD_TB.to_csv(output_file, encoding='utf-8', index=False, header=True, sep='\t')

print(f"Archivo guardado: {output_file}")

Procesado: /home/laura/andrea/scripts/Samples_Script_Test/SRR30362897/snps.all.ivar.tsv -> SNPs añadidos a BBDD_TB
Procesado: /home/laura/andrea/scripts/Samples_Script_Test/SRR30362892/snps.all.ivar.tsv -> SNPs añadidos a BBDD_TB
Procesado: /home/laura/andrea/scripts/Samples_Script_Test/SRR30362890/snps.all.ivar.tsv -> SNPs añadidos a BBDD_TB
Procesado: /home/laura/andrea/scripts/Samples_Script_Test/SRR30362893/snps.all.ivar.tsv -> SNPs añadidos a BBDD_TB
Procesado: /home/laura/andrea/scripts/Samples_Script_Test/SRR30362891/snps.all.ivar.tsv -> SNPs añadidos a BBDD_TB
Procesado: /home/laura/andrea/scripts/Samples_Script_Test/SRR30362896/snps.all.ivar.tsv -> SNPs añadidos a BBDD_TB
Procesado: /home/laura/andrea/scripts/Samples_Script_Test/SRR30362888/snps.all.ivar.tsv -> SNPs añadidos a BBDD_TB
Procesado: /home/laura/andrea/scripts/Samples_Script_Test/SRR30362895/snps.all.ivar.tsv -> SNPs añadidos a BBDD_TB
Procesado: /home/laura/andrea/scripts/Samples_Script_Test/SRR30362889/snps.all.i